In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import re
import pandas as pd

def scrape_reviews(url):
    # Configure ChromeOptions to use ChromeDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run ChromeDriver in headless mode (without opening the browser window)
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # Use ChromeDriverManager to automatically download the appropriate ChromeDriver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

    # Open the URL
    driver.get(url)

   
   

    # If the page still contains Loading... wait for the reviews to load
    waiting_time = 10
    while driver.page_source.find('Loading...') != -1:
        WebDriverWait(driver, waiting_time).until_not(EC.presence_of_element_located((By.CLASS_NAME, 'Loading')))
        #WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'ReviewsList')))
        source = driver.page_source
        waiting_time += 5


    reviews_element = driver.find_element(By.CLASS_NAME, 'ReviewsList')
    # Wait for the reviews to load
    #WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'ReviewsList')))

    reviews_element_html = reviews_element.get_attribute('innerHTML')
    #print(reviews_element_html)

    # Get the ReviewCards from the reviews element
    review_cards = reviews_element.find_elements(By.CLASS_NAME, 'ReviewCard')

    # #Get the number of reviews
    # number_of_reviews = len(review_cards)
    # print(number_of_reviews)
    # print("-------------------------------------------------------------------")
    # #Reviews Cards are lists of elements
    # reviews_cards_html = [review_card.get_attribute('innerHTML') for review_card in review_cards]

    # print(reviews_cards_html)


    # print("-------------------------------------------------------------------")
    

    #<section class="ReviewerProfile__avatar"><a class="Avatar Avatar--xsmall" href="https://www.goodreads.com/user/show/6611643-aline" as="div"><img data-testid="image" class="Avatar__image" alt="Profile Image for Aline." src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/users/1451466588i/6611643._UY200_CR33,0,200,200_.jpg"></a></section>
    #Get the reviews from the ReviewCards
    reviews = []
    for review_card in review_cards:
        #Get the reviewer name
        reviewer_name = review_card.find_element(By.CLASS_NAME, 'ReviewerProfile__name').text
        #Get the review text
        review_text = review_card.find_element(By.CLASS_NAME, 'TruncatedContent').text
        #Get the review rating
        #<section class="ReviewCard__row"><div class="ShelfStatus"><span aria-label="Rating 5 out of 5" role="img" class="RatingStars RatingStars__small"><span class="baseClass RatingStar--small"><svg viewBox="0 0 24 24" role="presentation"><path class="RatingStar__fill" d="M24 9.63469C24 9.35683 23.7747 9.13158 23.4969 9.13158H15.0892L12.477 1.34327C12.4269 1.19375 12.3095 1.0764 12.16 1.02625C11.8966 0.937894 11.6114 1.07983 11.523 1.34327L8.91088 9.13158H0.503157C0.33975 9.13158 0.186521 9.21094 0.0922364 9.3444C-0.0680877 9.57134 -0.0140806 9.88529 0.212865 10.0456L7.00408 14.8432L4.40172 22.6166C4.35092 22.7683 4.37534 22.9352 4.46749 23.066C4.6275 23.2932 4.94137 23.3476 5.16853 23.1876L12 18.3758L18.8317 23.183C18.9625 23.2751 19.1293 23.2994 19.281 23.2486C19.5445 23.1604 19.6865 22.8752 19.5983 22.6117L16.996 14.8432L23.7872 10.0456C23.9206 9.95133 24 9.7981 24 9.63469Z"></path></svg></span><span class="baseClass RatingStar--small"><svg viewBox="0 0 24 24" role="presentation"><path class="RatingStar__fill" d="M24 9.63469C24 9.35683 23.7747 9.13158 23.4969 9.13158H15.0892L12.477 1.34327C12.4269 1.19375 12.3095 1.0764 12.16 1.02625C11.8966 0.937894 11.6114 1.07983 11.523 1.34327L8.91088 9.13158H0.503157C0.33975 9.13158 0.186521 9.21094 0.0922364 9.3444C-0.0680877 9.57134 -0.0140806 9.88529 0.212865 10.0456L7.00408 14.8432L4.40172 22.6166C4.35092 22.7683 4.37534 22.9352 4.46749 23.066C4.6275 23.2932 4.94137 23.3476 5.16853 23.1876L12 18.3758L18.8317 23.183C18.9625 23.2751 19.1293 23.2994 19.281 23.2486C19.5445 23.1604 19.6865 22.8752 19.5983 22.6117L16.996 14.8432L23.7872 10.0456C23.9206 9.95133 24 9.7981 24 9.63469Z"></path></svg></span><span class="baseClass RatingStar--small"><svg viewBox="0 0 24 24" role="presentation"><path class="RatingStar__fill" d="M24 9.63469C24 9.35683 23.7747 9.13158 23.4969 9.13158H15.0892L12.477 1.34327C12.4269 1.19375 12.3095 1.0764 12.16 1.02625C11.8966 0.937894 11.6114 1.07983 11.523 1.34327L8.91088 9.13158H0.503157C0.33975 9.13158 0.186521 9.21094 0.0922364 9.3444C-0.0680877 9.57134 -0.0140806 9.88529 0.212865 10.0456L7.00408 14.8432L4.40172 22.6166C4.35092 22.7683 4.37534 22.9352 4.46749 23.066C4.6275 23.2932 4.94137 23.3476 5.16853 23.1876L12 18.3758L18.8317 23.183C18.9625 23.2751 19.1293 23.2994 19.281 23.2486C19.5445 23.1604 19.6865 22.8752 19.5983 22.6117L16.996 14.8432L23.7872 10.0456C23.9206 9.95133 24 9.7981 24 9.63469Z"></path></svg></span><span class="baseClass RatingStar--small"><svg viewBox="0 0 24 24" role="presentation"><path class="RatingStar__fill" d="M24 9.63469C24 9.35683 23.7747 9.13158 23.4969 9.13158H15.0892L12.477 1.34327C12.4269 1.19375 12.3095 1.0764 12.16 1.02625C11.8966 0.937894 11.6114 1.07983 11.523 1.34327L8.91088 9.13158H0.503157C0.33975 9.13158 0.186521 9.21094 0.0922364 9.3444C-0.0680877 9.57134 -0.0140806 9.88529 0.212865 10.0456L7.00408 14.8432L4.40172 22.6166C4.35092 22.7683 4.37534 22.9352 4.46749 23.066C4.6275 23.2932 4.94137 23.3476 5.16853 23.1876L12 18.3758L18.8317 23.183C18.9625 23.2751 19.1293 23.2994 19.281 23.2486C19.5445 23.1604 19.6865 22.8752 19.5983 22.6117L16.996 14.8432L23.7872 10.0456C23.9206 9.95133 24 9.7981 24 9.63469Z"></path></svg></span><span class="baseClass RatingStar--small"><svg viewBox="0 0 24 24" role="presentation"><path class="RatingStar__fill" d="M24 9.63469C24 9.35683 23.7747 9.13158 23.4969 9.13158H15.0892L12.477 1.34327C12.4269 1.19375 12.3095 1.0764 12.16 1.02625C11.8966 0.937894 11.6114 1.07983 11.523 1.34327L8.91088 9.13158H0.503157C0.33975 9.13158 0.186521 9.21094 0.0922364 9.3444C-0.0680877 9.57134 -0.0140806 9.88529 0.212865 10.0456L7.00408 14.8432L4.40172 22.6166C4.35092 22.7683 4.37534 22.9352 4.46749 23.066C4.6275 23.2932 4.94137 23.3476 5.16853 23.1876L12 18.3758L18.8317 23.183C18.9625 23.2751 19.1293 23.2994 19.281 23.2486C19.5445 23.1604 19.6865 22.8752 19.5983 22.6117L16.996 14.8432L23.7872 10.0456C23.9206 9.95133 24 9.7981 24 9.63469Z"></path></svg></span></span></div><span class="Text Text__body3"><a href="https://www.goodreads.com/review/show/3121973347">January 11, 2020</a></span></section>
        #Get the review rating from the aria-label
        rating_content = review_card.find_element(By.CLASS_NAME, 'ReviewCard__content').get_attribute('innerHTML')
        #review_rating = re.findall(r'\d+', rating_content)
        
        review_rating = re.findall(r'\d+', rating_content)[0]

        #print(review_rating)
     


        #Get the reviewer avatar
        avatar_content = review_card.find_element(By.CLASS_NAME, 'ReviewerProfile__avatar').get_attribute('innerHTML')
        reviewer_avatar_matches = re.findall(r'src="(.*?)"', avatar_content)
        if reviewer_avatar_matches:
            reviewer_avatar = reviewer_avatar_matches[0]
        else:
            reviewer_avatar = None


        review = {
            'user': reviewer_name,
            'avatar': reviewer_avatar,
            'text': review_text,
            'rating': review_rating
        }
        reviews.append(review)

    #reviews = sorted(reviews, key=lambda k: k['rating'], reverse=True)
    #print(reviews)
    
    # Save reviews to a dataframe
    reviews_df = pd.DataFrame(reviews)
    print(reviews_df)
    driver.quit()

    #return reviews

usage: ipykernel_launcher.py [-h] [--book_ids_path BOOK_IDS_PATH]
                             [--output_directory_path OUTPUT_DIRECTORY_PATH]
                             [--browser BROWSER]
                             [--rating_filter RATING_FILTER]
                             [--sort_order SORT_ORDER] [--format FORMAT]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"6f88ea74-0653-473d-bcf3-f20250ca6c8d" --shell=9002 --transport="tcp" --iopub=9004


SystemExit: 2

In [1]:
from bs4 import BeautifulSoup

import requests

#Regex usage
import re

import time

### 1.  Get the book reviews section
def get_book_reviews_section(book_show_url):
   
    # Get the book page
    book_page = requests.get(book_show_url)
    # Create a BeautifulSoup object
    soup = BeautifulSoup(book_page.content, 'html.parser')

    #beautified_html = soup.prettify()
    # Get the reviews section
    reviews_content = soup.find_all('div', class_='ReviewsList')

    return  reviews_content

In [2]:
def get_review_cards(reviews_content):

    # Convert the reviews_content to a BeautifulSoup object
    review_cards = []
    reviewCards = reviews_content.find_all('article', class_='ReviewCard')
    for card in reviewCards:
        review_cards.append(card)
    return review_cards

In [68]:
## 3. Get all the reviews

def get_all_reviews(review_cards):

    reviews = []

  

    for card in review_cards:

        if isinstance(card, BeautifulSoup):
            # Get the reviewer name
            reviewer_name = card.select_one('div.ReviewerProfile__name').get_text()

            # Get the review text
            review_text = card.find('div', class_='TruncatedContent').get_text()

            # Append the reviewer name and review text as a dictionary
            reviews.append({
                'reviewer_name': reviewer_name,
                'review_text': review_text
            })
        # #Get the reviewer name
        # reviewer_name = card.select_one('div.ReviewerProfile__name').get_text()

        # #reviewer_name = card.find('div', class_='ReviewerProfile__name').get_text()

        # #Get the review text
        # review_text = card.find('div', class_='TruncatedContent').get_text()
        # #Get the review rating
        # #review_content = card.find('span', class_='RatingStars RatingStars__small').get_text()
      
      
        # #Get the review rating
        # reviewRating = card.find('span', {'class': 'RatingStars RatingStars__small'})
        # #Check if the review has a rating
        # if reviewRating is None:
        #     return None
        # ratingString = reviewRating['aria-label']  # extract the aria-label value
        # rating = re.findall(r'\d+', ratingString)  # extract the number from the string
        
        # #Return the first element of the list
        # review_rating = rating[0]

            review = {
                'user': reviewer_name,
                'text': review_text,
                #'rating': review_rating
            }
            
            reviews.append(review)

    return reviews

In [ ]:
book_show_url = 'https://www.goodreads.com/book/show/'

books_ids = [
    '260487',
    '8497842',
    '2740909',
    '618259',
    '43352954'
]


for book_id in books_ids:
    url = book_show_url + book_id + '/reviews'
    reviews_content = get_book_reviews_section(url)
    #if the resullt is not in a beautiful soup object, convert it
    if not isinstance(reviews_content, BeautifulSoup):
        reviews_content = BeautifulSoup(str(reviews_content), 'html.parser')

    review_cards = get_review_cards(reviews_content)
    print(type(review_cards))
    # if not isinstance(review_cards, BeautifulSoup):
    #     review_cards = BeautifulSoup(str(review_cards), 'html.parser')
    # reviews = get_all_reviews(review_cards)
    # print(reviews)
    print(review_cards)
    print("--------------------------------------------------------------------------------")

In [ ]:
def get_top_reviews(book_id, reviews_count):
    url = f"https://www.goodreads.com/book/show/{book_id}/reviews"
    reviews_df = scrape_reviews(url)
    if reviews_df is None:
        return None
    if len(reviews_df) < reviews_count:
        return reviews_df.sort_values(by=['rating'], ascending=False)
    return reviews_df.sort_values(by=['rating'], ascending=False).head(reviews_count)

In [ ]:
# Usage example

book_id = 2740909
url = f'https://www.goodreads.com/book/show/{book_id}/reviews'

scrape_reviews(url)

In [ ]:
top_reviews = get_top_reviews(book_id, 10)